In [1]:
%pip install datasets evaluate
%pip install transformers[torch]
%pip install accelerate -U 
%pip3 install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pip3` not found.


## 1. Load data from csv file

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "train.csv"
)
print(df.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [6]:
# Shuffling and dropping unnecessary columns
df_shuffled = df.sample(frac=1, random_state=42)

# Dropping id, keyword and location columns as these columns consists of mostly nan values
# we will be using only text and target columns
df_shuffled.drop(["id", "keyword", "location"], axis=1, inplace=True)
df_shuffled.reset_index(inplace=True, drop=True)
print(df_shuffled.head())

                                                text  target
0  So you have a new weapon that can cause un-ima...       1
1  The f$&amp;@ing things I do for #GISHWHES Just...       0
2  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...       1
3  Aftershock back to school kick off was great. ...       0
4  in response to trauma Children of Addicts deve...       0


In [7]:
test_df = df_shuffled.sample(frac=0.1, random_state=42)
train_df = df_shuffled.drop(test_df.index)
print(f"Using {len(train_df)} samples for training and {len(test_df)} for validation")

Using 6852 samples for training and 761 for validation


In [8]:
train_df = train_df.to_numpy()
test_df  = test_df.to_numpy()

In [9]:
train_df[0][0]

'So you have a new weapon that can cause un-imaginable destruction.'

## 2. Process data

### 2.1 Load pre-trained tokenizer

In [10]:
from transformers import AutoTokenizer

pretrain_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

### 2.2 Buid dataset

In [11]:
import torch
from torch.utils.data import Dataset


class Tweets_Dataset(Dataset):
    def __init__(self, data, max_len,  tokenizer):
        super().__init__()
        self.data = data
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        text = self.data[idx][0]
        label = self.data[idx][1]

        text_processed = self.tokenizer.encode(text, max_length=self.max_len, padding='max_length')

        return {
            "input_ids": text_processed,
            "labels": label
        }

In [12]:
max_length = 200
train_data = Tweets_Dataset(train_df, max_length, pretrain_tokenizer)
test_data = Tweets_Dataset(test_df, max_length, pretrain_tokenizer)

## 3. Model

In [13]:
id2label = {0: "POSITIVE", 1: "NEGATIVE"}
label2id = {"POSITIVE": 0, "NEGATIVE": 1}

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## 4. Training

### 4.1 Eval function

In [16]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### 4.2 Train model

In [17]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer = pretrain_tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.410340,0.819974
2,0.448100,0.419809,0.830486
3,0.320300,0.452259,0.822602
4,0.234100,0.545349,0.818660
5,0.176700,0.664652,0.809461
6,0.138900,0.859782,0.814717
7,0.106800,0.882587,0.813403
8,0.106800,0.834886,0.822602
9,0.079600,0.952348,0.817346
10,0.068000,1.176226,0.816032


TrainOutput(global_step=8580, training_loss=0.10860769550005595, metrics={'train_runtime': 1615.1965, 'train_samples_per_second': 84.844, 'train_steps_per_second': 5.312, 'total_flos': 7091145434304000.0, 'train_loss': 0.10860769550005595, 'epoch': 20.0})

In [18]:
trainer.evaluate(test_data)

{'eval_loss': 0.41033995151519775,
 'eval_accuracy': 0.8199737187910644,
 'eval_runtime': 2.696,
 'eval_samples_per_second': 282.267,
 'eval_steps_per_second': 17.804,
 'epoch': 20.0}